# CMMA Inverse Strategy with Transaction Costs

This notebook explores a strategy that takes long positions above the fair price (2000) and short positions below the fair price, which is the opposite of traditional mean reversion. We'll include transaction costs in our analysis and optimize various parameters including CMMA thresholds.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the parent directory to the path to import utility functions
sys.path.append(os.path.abspath('..'))
from util import get_price_data, calculate_cmma_indicators

# Set plot style
plt.style.use('ggplot')
%matplotlib inline

## 1. Load and Preprocess Data

In [ ]:
# Load price data for squid_inksquid_data = get_price_data('squid_ink')squid_vwap = squid_data['vwap']squid_vwap = squid_vwap[:20000]  # Use in-sample data (first 20000 timestamps)# Calculate returnsreturns = squid_vwap.pct_change().fillna(0)# Plot the price dataplt.figure(figsize=(12, 6))plt.plot(squid_vwap)plt.axhline(y=2000, color='r', linestyle='--', label='Fair Price (2000)')plt.title('Squid_Ink VWAP Price')plt.xlabel('Timestamp')plt.ylabel('Price')plt.legend()plt.grid(True)plt.show()

## 2. Calculate CMMA Indicators

In [ ]:
# Calculate CMMA indicators for different lookback periodscmma_indicators = calculate_cmma_indicators(squid_vwap)# Plot CMMA indicatorsplt.figure(figsize=(12, 8))plt.subplot(2, 1, 1)plt.plot(squid_vwap)plt.axhline(y=2000, color='r', linestyle='--', label='Fair Price (2000)')plt.title('Squid_Ink VWAP Price')plt.legend()plt.grid(True)plt.subplot(2, 1, 2)for lookback in [10, 20, 30]:    plt.plot(cmma_indicators[f'CMMA_{lookback}'], label=f'CMMA_{lookback}')plt.axhline(y=0.7, color='r', linestyle='--', label='Upper Threshold (0.7)')plt.axhline(y=0.3, color='g', linestyle='--', label='Lower Threshold (0.3)')plt.title('CMMA Indicators')plt.legend()plt.grid(True)plt.tight_layout()plt.show()

## 3. Implement Inverse CMMA Dynamic Position StrategyThis strategy takes long positions above the fair price (2000) and short positions below the fair price, which is the opposite of traditional mean reversion.

In [ ]:
def cmma_inverse_position_strategy(prices, cmma, fair_price, upper_threshold, lower_threshold,                                 price_range, exponent, max_position, update_frequency):    """    Implements a dynamic position strategy based on CMMA indicators with an inverse approach.    Takes long positions above fair price and short positions below fair price.        Parameters:        prices (pd.Series): Series of prices        cmma (pd.Series): Series of CMMA indicator values        fair_price (float): Fair price level        upper_threshold (float): Upper threshold for CMMA        lower_threshold (float): Lower threshold for CMMA        price_range (float): Base price range for scaling        exponent (float): Exponent for scaling        max_position (float): Maximum position size        update_frequency (int): Update frequency for positions            Returns:        pd.Series: Series of positions    """    # Initialize positions    positions = pd.Series(0.0, index=prices.index)        # Initialize current position and last update index    current_position = 0.0    last_update_idx = 0        # Loop through prices    for idx in range(len(prices)):        # Check if we need to update the position        if idx - last_update_idx >= update_frequency:            # Get current price and CMMA value            current_price = prices.iloc[idx]            current_cmma = cmma.iloc[idx]                        # Calculate price deviation from fair price            price_deviation = current_price - fair_price                        # Determine position based on CMMA and price deviation            if current_cmma > upper_threshold and price_deviation > 0:                # CMMA is high and price is above fair price - go long                # Calculate position size based on price deviation                position_size = min(max_position, (abs(price_deviation) / price_range) ** exponent)                current_position = position_size            elif current_cmma < lower_threshold and price_deviation < 0:                # CMMA is low and price is below fair price - go short                # Calculate position size based on price deviation                position_size = min(max_position, (abs(price_deviation) / price_range) ** exponent)                current_position = -position_size            else:                # No clear signal - maintain current position                pass                            # Update last update index            last_update_idx = idx                # Set the current position        positions.iloc[idx] = current_position        return positions

## 3. Implement Inverse CMMA Dynamic Position StrategyThis strategy takes long positions above the fair price (2000) and short positions below the fair price, which is the opposite of traditional mean reversion.

In [ ]:
def cmma_inverse_position_strategy(prices, cmma, fair_price, upper_threshold, lower_threshold,                                 price_range, exponent, max_position, update_frequency):    """    Implements a dynamic position strategy based on CMMA indicators with an inverse approach.    Takes long positions above fair price and short positions below fair price.        Parameters:        prices (pd.Series): Series of prices        cmma (pd.Series): Series of CMMA indicator values        fair_price (float): Fair price level        upper_threshold (float): Upper threshold for CMMA        lower_threshold (float): Lower threshold for CMMA        price_range (float): Base price range for scaling        exponent (float): Exponent for scaling        max_position (float): Maximum position size        update_frequency (int): Update frequency for positions            Returns:        pd.Series: Series of positions    """    # Initialize positions    positions = pd.Series(0.0, index=prices.index)        # Initialize current position and last update index    current_position = 0.0    last_update_idx = 0        # Loop through prices    for idx in range(len(prices)):        # Check if we need to update the position        if idx - last_update_idx >= update_frequency:            # Get current price and CMMA value            current_price = prices.iloc[idx]            current_cmma = cmma.iloc[idx]                        # Calculate price deviation from fair price            price_deviation = current_price - fair_price                        # Determine position based on CMMA and price deviation            if current_cmma > upper_threshold and price_deviation > 0:                # CMMA is high and price is above fair price - go long                # Calculate position size based on price deviation                position_size = min(max_position, (abs(price_deviation) / price_range) ** exponent)                current_position = position_size            elif current_cmma < lower_threshold and price_deviation < 0:                # CMMA is low and price is below fair price - go short                # Calculate position size based on price deviation                position_size = min(max_position, (abs(price_deviation) / price_range) ** exponent)                current_position = -position_size            else:                # No clear signal - maintain current position                pass                            # Update last update index            last_update_idx = idx                # Set the current position        positions.iloc[idx] = current_position        return positions